# Mortalità per territorio di evento in Italia

##### WebScraping

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date

In [4]:
url = 'http://dati.istat.it/Index.aspx?QueryId=26399#'
request = requests.get(url).text
soup = BeautifulSoup(request, 'html.parser')

In [5]:
soup.find_all(name='tr')

[<tr><th class="PDimName" colspan="2"><b><a class="dimensionSelectMemberWhiteButton" href="#" onclick="OpenDialog('./ModalDimWebTree.aspx?DimensionCode=TIPO_DATO15', 'Customize');">Tipo dato</a></b></th><th align="left" class="PDim" colspan="3">morti</th></tr>,
 <tr><th class="PDimName" colspan="2"><b><a class="dimensionSelectMemberWhiteButton" href="#" onclick="OpenDialog('./ModalDimWebTree.aspx?DimensionCode=ITTER107', 'Customize');">Territorio</a></b></th><th align="left" class="PDim" colspan="3"><select class="PDim" id="PDim_ITTER107" name="PDim_ITTER107" onchange="ChgPageSel(this)"><option selected="" value="0~IT">Italia</option><option value="1~ITC">  Nord-ovest</option><option value="2~ITC1">    Piemonte</option><option value="3~ITC11">      Torino</option><option value="4~ITC12">      Vercelli</option><option value="5~ITC15">      Novara</option><option value="6~ITC16">      Cuneo</option><option value="7~ITC17">      Asti</option><option value="8~ITC18">      Alessandria</opti

In [31]:
table_row=soup.find_all(name='tr')
list_disease=[]

def replace_float(n):
    try:
        new_value = n.replace("\xa0", "") 
        value_float = float(new_value.strip())
        return (value_float)
    except ValueError:
        return print(f"Expression '{n}' is not a valid number.")

    
for i,row in enumerate(table_row):
    if (i==6):
        head = row.find('a').string
        print(head)
    if (i>6 and i<len(table_row)-3):
        row_td= row.find_all('td')
        dict_disease={}
        qty_male, qty_female, qty_total = "", "", ""
        flag_subtree="N"
        spans=row.find_all('span')
        
        for j, td in enumerate(row_td):
            td_text = td.get_text(strip=True)
            for span in spans:
                img = span.find('img')
            if(j==0 and img):
                type_disease=str(td_text)
                flag_subtree="Y"
            elif(j==0):
                type_disease=str(td_text)
            elif(j==2):
                qty_male=replace_float(td_text)
            elif(j==3):
                qty_female=replace_float(td_text)
            elif(j==4):
                qty_total=replace_float(td_text) 
            
            dict_disease['type_disease']=type_disease
            dict_disease['qty_male']=qty_male
            dict_disease['qty_female']=qty_female
            dict_disease['qty_total']=qty_total
            dict_disease['flag_subtree']=flag_subtree
        list_disease.append(dict_disease)

Causa iniziale di morte - European Short List
Expression '..' is not a valid number.
Expression '..' is not a valid number.
Expression '..' is not a valid number.
Expression '..' is not a valid number.
Expression '..' is not a valid number.


In [32]:
list_disease

[{'type_disease': 'alcune malattie infettive e parassitarie',
  'qty_male': 6409.0,
  'qty_female': 7377.0,
  'qty_total': 13786.0,
  'flag_subtree': 'Y'},
 {'type_disease': 'tubercolosi',
  'qty_male': 140.0,
  'qty_female': 88.0,
  'qty_total': 228.0,
  'flag_subtree': 'N'},
 {'type_disease': 'aids (malattia da hiv)',
  'qty_male': 305.0,
  'qty_female': 84.0,
  'qty_total': 389.0,
  'flag_subtree': 'N'},
 {'type_disease': 'epatite virale',
  'qty_male': 883.0,
  'qty_female': 853.0,
  'qty_total': 1736.0,
  'flag_subtree': 'N'},
 {'type_disease': 'altre malattie infettive e parassitarie',
  'qty_male': 5081.0,
  'qty_female': 6352.0,
  'qty_total': 11433.0,
  'flag_subtree': 'N'},
 {'type_disease': 'tumori',
  'qty_male': 97867.0,
  'qty_female': 79991.0,
  'qty_total': 177858.0,
  'flag_subtree': 'Y'},
 {'type_disease': 'tumori maligni',
  'qty_male': 92777.0,
  'qty_female': 75436.0,
  'qty_total': 168213.0,
  'flag_subtree': 'Y'},
 {'type_disease': 'di cui tumori maligni delle la

In [52]:
df = pd.DataFrame(list_disease)
df = df.drop(df.index[-1])
df

,type_disease,qty_male,qty_female,qty_total,flag_subtree
0,alcune malattie infettive e parassitarie,6409.0,7377.0,13786.0,Y
1,tubercolosi,140.0,88.0,228.0,N
2,aids (malattia da hiv),305.0,84.0,389.0,N
3,epatite virale,883.0,853.0,1736.0,N
4,altre malattie infettive e parassitarie,5081.0,6352.0,11433.0,N
...,...,...,...,...,...
85,di cui altri accidenti,5105.0,6752.0,11857.0,N
86,suicidio e autolesione intenzionale,2923.0,789.0,3712.0,N
87,"omicidio, aggressione",135.0,91.0,226.0,N
88,eventi di intento indeterminato,8.0,4.0,12.0,N


#####  Loggin function

In [65]:
def log_function(message):
    data_format='%d-%h-%Y  %H:%M:%S'
    data=datetime.now()
    parsed_data=data.strftime(data_format)
    
    with open('log_file.txt','a') as file:
        file.write(parsed_data+', '+message+'\n')

In [53]:
# Testing the log file
log_function("Test")

#####  Load

In [67]:
def load_data(file_data, target_file_name=None):
    try:
        if(target_file_name!=None):
            file_data.to_csv(target_file_name, index=False)
            log_function(f'Load data into csv file {target_file_name}')
        else:
            data=datetime.now().date()
            target_name="mortalita_per_territorio_"+str(data)+".csv"
            file_data.to_csv(target_name, index=False)
            log_function(f'Load data into csv file {target_name}')
    except Exception as e:
        print(f"An unexpected error has occurred {e}")

In [69]:
# Insert dataframe into a csv file
load_data(df)

In [61]:
# Read the csv file 
pd.read_csv('mortalita_per_territorio_2023-09-19.csv')

,type_disease,qty_male,qty_female,qty_total,flag_subtree
0,alcune malattie infettive e parassitarie,6409.0,7377.0,13786.0,Y
1,tubercolosi,140.0,88.0,228.0,N
2,aids (malattia da hiv),305.0,84.0,389.0,N
3,epatite virale,883.0,853.0,1736.0,N
4,altre malattie infettive e parassitarie,5081.0,6352.0,11433.0,N
...,...,...,...,...,...
85,di cui altri accidenti,5105.0,6752.0,11857.0,N
86,suicidio e autolesione intenzionale,2923.0,789.0,3712.0,N
87,"omicidio, aggressione",135.0,91.0,226.0,N
88,eventi di intento indeterminato,8.0,4.0,12.0,N


In [68]:
# Insert data with flag subtree = yes
df_flag = df[df['flag_subtree']=='Y']
load_data(df_flag, 'mortalita_per_territorio_FY_2023-09-19.csv')

In [58]:
df_flag

,type_disease,qty_male,qty_female,qty_total,flag_subtree
0,alcune malattie infettive e parassitarie,6409.0,7377.0,13786.0,Y
5,tumori,97867.0,79991.0,177858.0,Y
6,tumori maligni,92777.0,75436.0,168213.0,Y
31,"malattie endocrine, nutrizionali e metaboliche",15225.0,18360.0,33585.0,Y
34,disturbi psichici e comportamentali,8850.0,18121.0,26971.0,Y
39,malattie del sistema nervoso e degli organi di...,14539.0,18625.0,33164.0,Y
43,malattie del sistema circolatorio,98850.0,128500.0,227350.0,Y
44,malattie ischemiche del cuore,34095.0,29857.0,63952.0,Y
50,malattie del sistema respiratorio,30623.0,26490.0,57113.0,Y
53,malattie croniche delle basse vie respiratorie,13677.0,10549.0,24226.0,Y
